In [1]:
import pandas as pd
import urllib.request
from bs4 import BeautifulSoup

In [2]:
# Create an empty list to store extracted data
l = []

In [3]:
# Crawling through the first page with 16 listings
page = urllib.request.urlopen('https://www.engelvoelkers.com/en/search/?q=&startIndex=0&businessArea=residential&sortOrder=DESC&sortField=sortPrice&pageSize=18&facets=bsnssr%3Aresidential%3Bcntry%3Ahong_kong%3Btyp%3Arent%3B')
soup = BeautifulSoup(page, 'html.parser')
all_first = soup.find_all("div", {"class": "ev-teaser-content"})
for i in all_first:
    d = {}
    d["Place"] = i.find("div", {"class": "ev-teaser-title"}).text
    d["Price"] = i.find("div", {"class": "ev-value"}).text.replace(" HKD","").replace(".","")
    if i.find("img", {"title": "Bedrooms"}):
        d["Bedroom"] = float(i.find("img", {"title": "Bedrooms"}).find_next("span").text.replace(" ",""))
    if i.find("img", {"title": "Bathrooms"}):
        d["Bathroom"] = float(i.find("img", {"title": "Bathrooms"}).find_next("span").text.replace(" ",""))
    if i.find("img", {"title": "Saleable Area approx."}):
        d["Saleable Area"] = float(i.find("img", {"title": "Saleable Area approx."}).find_next("span").text.replace(",","").replace(" m²",""))
    d["Area"] = i.find("div", {"class": "ev-teaser-subtitle"}).text.split(",")[2]
    d["Region"] = i.find("div", {"class": "ev-teaser-subtitle"}).text.split(",")[1]
    l.append(d)

In [4]:
# All subsequent pages with 18 listings
# Note that the last page may have fewer than 18 listings, however, since we are not going to any page after that, it is acceptable to run the same code without errors
base_url = 'https://www.engelvoelkers.com/en/search/?q=&startIndex=' 
base2_url = '&businessArea=residential&sortOrder=DESC&sortField=sortPrice&pageSize=18&facets=bsnssr%3Aresidential%3Bcntry%3Ahong_kong%3Btyp%3Arent%3B'
for page in range(16, 942, 18):
    #print(base_url+str(page)+base2_url)
    r = urllib.request.urlopen(base_url+str(page)+base2_url)
    soup = BeautifulSoup(r, 'html.parser')
    all_pages = soup.find_all("div", {"class": "ev-teaser-content"})
    for i in all_pages:
        d = {}
        d["Place"] = i.find("div", {"class": "ev-teaser-title"}).text
        d["Price"] = i.find("div", {"class": "ev-value"}).text.replace(" HKD","").replace(".","")
        if i.find("img", {"title": "Bedrooms"}):
            d["Bedroom"] = float(i.find("img", {"title": "Bedrooms"}).find_next("span").text.replace(" ",""))
        if i.find("img", {"title": "Bathrooms"}):
            d["Bathroom"] = float(i.find("img", {"title": "Bathrooms"}).find_next("span").text.replace(" ",""))
        if i.find("img", {"title": "Saleable Area approx."}):
            d["Saleable Area"] = float(i.find("img", {"title": "Saleable Area approx."}).find_next("span").text.replace(" m²",""))
        d["Area"] = i.find("div", {"class": "ev-teaser-subtitle"}).text.split(",")[2]
        d["Region"] = i.find("div", {"class": "ev-teaser-subtitle"}).text.split(",")[1]
        l.append(d)

In [5]:
df = pd.DataFrame(l, columns = ["Place", "Price", "Bedroom", "Bathroom", "Saleable Area", "Area", "Region" ])
# remove listings where price is '(on request)' and convert to float for mathematical calculations 
df["Price"] = pd.to_numeric(df["Price"], errors='coerce')
df = df.dropna(subset = ["Price"])
df

,Place,Price,Bedroom,Bathroom,Saleable Area,Area,Region
0,10 SHOUSON HILL ROAD EAST 壽臣山道東 10 號,1000000.0,7.0,6.0,1030.11,Southern District/Repulse Bay,Hong Kong Island
1,7-15 Mount Kellett Road 加列山道7號,930000.0,11.0,12.0,1022.12,Peak/South,Hong Kong Island
3,7-15 Mount Kellett Road 加列山道7號,520000.0,6.0,7.0,529.55,Peak/South,Hong Kong Island
4,Tavistock 騰皇居,500000.0,4.0,3.0,488.11,Mid Level Central,Hong Kong Island
6,The Lily 淺水灣道129號,400000.0,5.0,3.0,491.74,Peak/South,Hong Kong Island
7,The Nautilus 南灣道16A號,380000.0,5.0,4.0,326.00,Peak/South,Hong Kong Island
8,SEVERN 8 倚巒,380000.0,4.0,5.0,332.69,Peak/South,Hong Kong Island
9,The Mount Austin The Mount Austin,368000.0,4.0,4.0,351.82,Peak/South,Hong Kong Island
10,Branksome Crest BRANKSOME CREST,350000.0,3.0,3.0,348.39,Sheung Wan/Central/Admiralty,Hong Kong Island
11,Cheuk Nang Lookout 卓能山莊,350000.0,4.0,3.0,412.21,Peak/South,Hong Kong Island


In [6]:
df.to_csv("Renting_data.csv", index = False)